In [ ]:
# Dependencies
import matplotlib.pyplot as plt
import numpy as np
from scipy.fft import fft,ifft
%matplotlib notebook

## Discrete Fourier tranform (DFT)
### Background information
The following notes are based on [Scipy Documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.fft.fft.html#scipy.fft.fft).

In this implementantion the DFT is defined as: 
\begin{equation}
y[k] = \sum_{n=0}^{N-1} x[n] exp\{-2\pi i\frac{kn}{N}\},
\end{equation}

and the Inverse Fourier Transform (IFT) is defined as: 
\begin{equation}
x[n] = \frac{1}{N}\sum_{n=0}^{N-1} y[k] exp\{2\pi i\frac{kn}{N}\}
\end{equation}

From the definition: 
\begin{equation}
y[0] = \sum_{n=0}^{N-1} x[n],
\end{equation}

that is, y[0] contains the the sum of the signal, which is purely real for real inputs. 
Then y[1:N/2] (for N even) contains the positive frequency terms and f[N/2+1:] contains the negative frequency terms. 

### Exercise 1.1

In [ ]:
# length of the signal
t_fin = 1
# sampling frequency
fs = 12      
# sampling points
sp = fs*t_fin
# time partition 
t = np.linspace(0,t_fin,sp) #[s]
# frequency
f = np.arange(0,sp/t_fin,1/t_fin)


# function
freq = 1 #[Hz]
omega = 2*np.pi*freq
func = np.sin(omega*t)

# fourier transform of func
ft = np.empty(sp, dtype = np.complex_)
ft = fft(func) # by default is operating with backward normalisation (i.e, not multiplying by any prefactor)

# inverse fourier transform of ft
ift = np.empty(sp, dtype = np.complex_)   # *1/sp
ift = ifft(ft)


fig, ax = plt.subplots(3)
ax[1].plot(f,ft.real)
ax[1].set_xlabel("Frequency [Hz]")
ax[1].set_ylabel(r"$Re(F[\nu])$")
ax[2].plot(f,ft.imag)
ax[2].set_xlabel("Frequency [Hz]")
ax[2].set_ylabel(r"$Im(F[\nu])$")
ax[0].plot(t,func,"-",color = "red",label=r"$sin(\omega t)$")
ax[0].plot(t,ift.real,".",color = "blue",label =r"$F^{-1}[F[\nu]]$")
ax[0].legend()
ax[0].set_xlabel("Time (s)")
ax[0].set_ylabel(r"$sin(\omega t)$")

plt.show()

### Exercise 1.2
> DEF: A Power Spectral Density (PSD) is the measure of signal's power content versus frequency.

From the definition of DFT and IFT reported in the previous paragraph and from the Paserval's theorem, we know that: 
\begin{equation}
\sum_{n=0}^{N-1} |x[n]|^2 = \frac{1}{N}\sum_{k=0}^{N-1} |y[k]|^2
\end{equation}
Therefore, by multipling both terms for $1/N$:
\begin{equation} 
\frac{1}{N}\sum_{n=0}^{N-1} |x[n]|^2 = P_{avg} = \frac{1}{N^2}\sum_{k=0}^{N-1} |y[k]|^2,
\end{equation}

therefore, for each DFT bin, we can define a local power density:
\begin{equation}
P(k) = \frac{1}{N^2}|y[k]|^2.
\end{equation}

This forms the power spectrum of the signal, expressed in $V^2/[Hz]$. This follows from: 

Otherwise, it is also to express the power spectrum as: 



In [ ]:
# Dependencies
import matplotlib.pyplot as plt
import numpy as np
from scipy.fft import fft,ifft
#%matplotlib notebook    #uncomment if you work on jupyter notebook

def power_spectrum(V_in,N):
    """
    Calculates the power spectrum in unit of V^2/Hz
    """
    V_out = np.empty(N,dtype = np.complex_)
    V_out = fft(V_in, axis=0)    
    return (abs(V_out)**2)*T/(2*(N/2)**2)
        
def dB(P_in, P_ref = 1):
    """
    Calculates the power spectrum in unit of dB
    """
    return 20*np.log10(abs(P_in)/P_ref)

def trapezoidal(f,xmin,xmax,N):
    delta_k = (xmax-xmin)/N
    Trap = 0.
    for k in range(1,N):
        Trap += delta_k*(f[k]+f[k-1])/2.0
    return Trap

def power_area(V_in):
    """
    Calculates the power spectrum in unit of V^2/Hz and in dB. 
    In addition to that, it calculates the area underneath the power spectrum in V^2/Hz.
    The returned area should be equal to the square value of V_rms of the signal.
    """
    P_V2s = power_spectrum(V_in,sp)
    area = trapezoidal(P_V2s,0,f[int(sp/2)],int(sp/2))
    P_db = dB(P_V2s,sp)
    return P_V2s,area, P_db

def tot_pow(P_in):
    acc = 0
    for P_k in P_in[0:int(len(P_in)/2)]:
        acc += P_k
    return acc/T

# sampling frequency
fs = 500 # Hz
# Period of time
T = 2 #s
# sampling point
sp = fs*T
# time array
t = np.linspace(0,T,sp)
# frequency array
f = np.arange(0,(sp)/T,1./T)

#----------------------------------------------------------------------------
# Signal 1: white noise
from numpy.random import normal

def white_noise(time,amp = 1):
    return amp*normal(0,1,len(time))

# noise parameters
Temp = 300   #K
R = 1e6   #ohm
df = 1e4 #Hz
kb = 1.38064852e-23 # m2 kg s-2 K-1 Boltzmann constant
Vnoise = np.sqrt(4*kb*Temp*R*df) #Johnson-Nyquist noise

V_1 = white_noise(t,Vnoise)
#---------------------------------------------------------------------------
# Signal 2: sin function
A2 = 12e-6 # V amplitude
freq2 = 80  # Hz
V_2 = A2*np.sin(2*np.pi*freq2*t)

#---------------------------------------------------------------------------
# Signal 3: sin function
A3 = 6e-6   # V
freq3 = 170 # Hz
V_3 = A3*np.sin(2*np.pi*freq3*t)
#---------------------------------------------------------------------------
# Calculating the power spectrum (V^2/Hz, dB) and the area underneath the signal

P_V2s_1, area_1, P_db_1 = power_area(V_1)
P_V2s_2, area_2, P_db_2 = power_area(V_2)
P_V2s_3, area_3, P_db_3 = power_area(V_3)

pow1 = tot_pow(P_V2s_1)
pow2 = tot_pow(P_V2s_2)
pow3 = tot_pow(P_V2s_3)
pow_th_comb = Vnoise**2+A2**2/2+A3**2/2  # theoretical value of the power for the three (uncorrelated) signals
 
#---------------------------------------------------------------------------
# Combining signals:
V_comb = V_1 + V_2 + V_3
P_V2s_comb, _ , _ = power_area(V_comb)
pow_comb = tot_pow(P_V2s_comb)
#---------------------------------------------------------------------------

print("Signal 1: V_rms^2:",Vnoise**2," Area:",area_1)
print("Signal 2: V_rms^2:", A2**2/2," Area:", area_2)
print("Signal 3: V_rms^2:", A3**2/2, "Area:", area_3)

print("Power of the first signal:", pow1)
print("Power of the second signal:", pow2)
print("Power of the third signal:", pow3)
print("Theorical power of the combined signal:", pow_th_comb, "Calculated power:", pow_comb)

fig1, ax1 = plt.subplots(3, sharex = True)
fig1.suptitle('Voltage signals (V)', fontsize=13)
ax1[0].plot(t, V_1)
ax1[1].plot(t, V_2)
ax1[2].plot(t,V_3)
ax1[2].set_xlabel("Time (s)")

fig2, ax2 = plt.subplots(3, sharex = True)
fig2.suptitle(r'Power spectra $(\frac{V^2}{Hz})$', fontsize=13)
ax2[0].plot(f[0:int(sp/2)], P_V2s_1[0:int(sp/2)])
ax2[1].plot(f[0:int(sp/2)], P_V2s_2[0:int(sp/2)])
ax2[2].plot(f[0:int(sp/2)], P_V2s_3[0:int(sp/2)])
ax2[2].set_xlabel("Frequency (Hz)")

fig3, ax3 = plt.subplots(3, sharex = True)
fig3.suptitle(r'Power spectra (dB)', fontsize=13)
ax3[0].plot(f[0:int(sp/2)], P_db_1[0:int(sp/2)])
ax3[1].plot(f[0:int(sp/2)], P_db_2[0:int(sp/2)])
ax3[2].plot(f[0:int(sp/2)], P_db_3[0:int(sp/2)])
ax3[2].set_xlabel("Frequency (Hz)")

fig4, ax4 = plt.subplots()
fig4.suptitle('Combined signal power spectrum' ,fontsize = 13)
ax4.plot(f[0:int(sp/2)], P_V2s_comb[0:int(sp/2)])
ax4.set_ylabel(r"Power $(\frac{V^2}{Hz})$")
ax4.set_xlabel("Frequency (Hz)")


#---------------------------------------------------------------------------------------
# Dependence on the sampling frequency and on the length of the signal

# sampling frequency
fs = [100,300,500,750,1000] # Hz
# length of the signal
T = [0.5,2.5,5,7.5,10]

fig5, ax5 = plt.subplots(len(fs),sharex = True)
fig5.suptitle('Dependence on the sampling frequency', fontsize=13)
ax5[len(fs)-1].set_xlabel("time (s)")
length = 2 #s
for i,freq in enumerate(fs):
    sp = int(freq*length)
    t = np.linspace(0,length,sp)
    V_3 = A3*np.sin(2*np.pi*freq3*t)
    ax5[i].plot(t,V_3, label= str(freq)+" Hz")
    ax5[i].axhline(-A3, color = "red")
    ax5[i].axhline(A3,color = "red")
    ax5[i].set_ylim(-7e-6,7e-6)
    ax5[i].set_xlim(0,0.3)
    ax5[i].legend(loc = "right")

fig6, ax6 = plt.subplots(len(T),sharex = True)
fig6.suptitle('Dependence on the signal length', fontsize=13)
ax6[len(T)-1].set_xlabel("time (s)")
freq = 500 # Hz
for i,length in enumerate(T):
    sp = int(freq*length)
    t = np.linspace(0,length,sp)
    V_3 = A3*np.sin(2*np.pi*freq3*t)
    ax6[i].plot(t,V_3, label= str(length)+" sec")
    ax6[i].axhline(-A3, color = "red")
    ax6[i].axhline(A3,color = "red")
    ax6[i].set_ylim(-7e-6,7e-6)
    ax6[i].set_xlim(0,0.3)
    ax6[i].legend(loc = "right")
    
plt.show()

### Exercise 1.3

In [ ]:
# Dependencies
import matplotlib.pyplot as plt
import numpy as np
from scipy.fft import fft,ifft
from scipy.optimize import curve_fit 
import pandas as pd


def PeaksFinder(arr,std):
    n = len(arr)
    peaks = []  # contains the values of arr in correspondence of the peaks
    pos = []   # contains the indeces of arr in correspondence of the peaks
    
    # First element 
    if (arr[0]-arr[1] >= 3*std):
        peaks.append(arr[0])
        pos.append(0)
    
    # from the second to the one before the last
    for i in range(1, n-1):
        if (arr[i] - arr[i-1] >= 3*std and arr[i] - arr[i+1]>= 3*std):
            peaks.append(arr[i])
            pos.append(i)
    # last element
    if (arr[n-1] - arr[n-2] >= 3*std):
        peaks.append(arr[n-1])
        pos.append(n-1)
    
    return pos, peaks


def modulus(freq,R,C):
    return R / np.sqrt(1+(2*np.pi*C*R*freq)**2)

def phase(freq,R,C):
    return np.arctan(-2*np.pi*freq*C*R)

#selecting the x-range to fit
def select_range(df,header,x_min,x_max):
    selected_df = df.loc[(df[header] >= x_min) & (df[header] <= x_max)]
    return selected_df

#Importing data
data = pd.read_excel("data_DAQ.xls")

# Interpreting and converting data into numpy arrays
time = np.array(data["time(s)"])
volt = np.array(data["input(V)"])
amp = np.array(data["output(A)"])

# sampling point
sp = len(time)
#Defining length of the signal (L) and sampling frequency (sf)
L = time[-1]-time[0]
sf = 1/(time[1]-time[0])

# Defining frequency array
f = np.arange(0,(sp)/L,1./L)

#Calculting DFT of volt and amp 
v_dft = fft(volt)
a_dft = fft(amp)

# Calculating stardard deviation of voltage and current in a flat interval
df_freq = pd.DataFrame(columns =["volt", "amp", "freq"])
df_freq["volt"] = v_dft
df_freq["amp"] = a_dft
df_freq["freq"] = f

x_min = 1e4
x_max = 1.8e4
df_flat = select_range(df_freq,"freq",x_min,x_max)

std_values = df_flat.std()  
v_std = std_values[0]
a_std = std_values[1]

# Finding the peaks position ror v_dft 
pos, _ = PeaksFinder(v_dft[0:int(sp/2)],v_std)


# calculating the impedence
Z = v_dft/a_dft
Z = Z[pos]

# Calculating phase and modulus of the impedance
Z_phase = np.arctan(Z.imag/Z.real)
Z_modulus = abs(Z)

#  Creating a Bode plot of the impedance
fig7, ax7 = plt.subplots(2,sharex = True)
fig7.suptitle(r'Bode plot', fontsize=12)
ax7[0].plot(f[pos],Z_modulus, label = "Modulus")
ax7[0].set_yscale('log')
ax7[0].set_ylabel(r'$log_{10}(|Z|)$')
ax7[0].set_xscale('log')
ax7[1].plot(f[pos],abs(Z_phase),label ="Phase")
ax7[1].set_yscale('log')
ax7[1].set_ylabel(r'$log_{10}(|atan(\frac{Im(Z)}{Re(Z)})|)$')
ax7[1].set_xscale('log')

# Defyining a frequency array for plotting
freq_arr = np.linspace(pos[0],20000,1000)

print(Z_phase)


# Fitting the phase of the impedance
par_phase, covs_phase = curve_fit(phase,f[pos], Z_phase,
                                               p0=[1e5,1e-9])


# Fitting the modulus of the impedance 
par_mod, covs_mod = curve_fit(modulus,f[pos], Z_modulus,
                                               p0=(1e5,1e-9))

print("Fit result of the phase of the impedance")
print("Resistance: ", par_phase[0],"Ohm","Capacitance: ",par_phase[1],"F")

print("Fit results of the modulus of the impedance")
print("Resistance: ",par_mod[0],"Ohm","Capacitance: ",par_mod[1],"F")

fit_phase = phase(par_phase[0],par_phase[1],freq_arr)
fit_modulus = modulus(freq_arr,par_mod[0],par_mod[1])

fig8,ax8 = plt.subplots(2)
ax8[0].plot(f[pos],Z_phase,".",markersize = 10 ,color = "red", label ="data_DAQ")
ax8[0].plot(freq_arr, fit_phase, color = "blue", label ="Fitting")
ax8[0].set_title("Impedance phase")
ax8[0].legend()
ax8[1].plot(f[pos],Z_modulus,".", markersize = 10,color = "red", label ="data_DAQ")
ax8[1].plot(freq_arr, fit_modulus, color = "blue", label ="Fitting")
ax8[1].set_title("Impedance modulus")
ax8[1].legend()
fig8.tight_layout()

plt.show()

# Exercise 1.4
## Windowing

In [11]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.fft import fft,ifft
%matplotlib notebook

def gaussian(x,y0,A,x0,width): 
    return y0 + A*np.exp(-(x-x0)**2/(2*width**2))


L = 1.25
# sampl. freq
fs = 100
# sampling period
T = 1/fs
# sam. points
sp = fs*int(L)
# time partition 
t = np.linspace(0,L,sp) #[s]
# frequency
freq = np.arange(0,sp/L,1/L)

# sine definition
f = 10
y = np.sin(2*np.pi*f*t)
y_nowind = fft(y)


# gaussian
window = gaussian(t,0,1.,L/2,L/6.)  #  3 sigma 

y_wind = fft(y*window)

fig, ax = plt.subplots(2)
ax[0].plot(f[0:int(sp/2)],y_nowind[0:int(sp/2)])
ax[1].plot(f[0:int(sp/2)],y_wind[0:int(sp/2)])



<IPython.core.display.Javascript object>

TypeError: 'int' object is not subscriptable